Neural Model Identification: \
Trains a neural network based on data from the physics model to use as the predictive model
Note: You need to install the matlab engine to be able to run matlab code from pythone

In [1]:
import matplotlib.pyplot as plt

In [2]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#from skopt.space import Real, Categorical, Integer
from sklearn import preprocessing

/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/gmakr/opt/anaconda3/e

In [3]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [4]:
import scipy.io
model_data = scipy.io.loadmat('Data.mat')

In [5]:
Zin=model_data.get("Zin")
Zout=model_data.get("Zout")

## Define Functions

In [6]:
def swish(x, beta=1):
    return x * sigmoid(beta * x)

def get_model_layers(nodes, layers, activation, learning_rate, training=None):
    if activation == 'swish':
        activation = swish
    else:
        activation = activation

    inputs = tf.keras.layers.Input(shape=(in_dim,))
  #  xadd = (np.array(inputs[:, :out_dim])).reshape(None,1)
   # print(inputs[:, :out_dim].shape)
    #print(xadd.shape)
    x = tf.keras.layers.Dense(units=nodes, activation=activation)(inputs)
    for i in range(layers - 1):  # at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation=activation)(x)

    #x = tf.keras.layers.Dropout(rate=0.1)(x, training=training)
    outputs = tf.keras.layers.Dense(out_dim, activation="linear")(x)
   # outputs = tf.keras.layers.Add()([outputs, inputs[:, 2:]])

    #outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]])
    # outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]]) NN4UQ
    #    out = keras.layers.Dense(units=3, activation= "linear")(added)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error')
    return model

In [7]:
get_custom_objects().update({'swish': Activation(swish)})

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(Zin, Zout, shuffle=True, train_size=0.9, random_state=42)
in_dim = X_train.shape[1]
out_dim = Y_train.shape[1]

In [9]:
learning_rate=0.05#gp_result.x[0]
num_layers = 3 #gp_result.x[1]
num_nodes = 5 #gp_result.x[2]
batch_size=10 #gp_result.x[3]
epochs=200
activation='swish'
DropModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                             activation=activation, learning_rate=learning_rate,
                             training=True)  # True-activates dropout during prediction
DropModel.summary()
DropModel.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data = (X_test, Y_test))

DetModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                            activation=activation, learning_rate=learning_rate)
#name = '{}_{}_{}_{}'.format(num_nodes, num_layers, activation, learning_rate)
#DropModel.save(root + 'results/nc/' + name)
DetModel.set_weights(DropModel.get_weights())
score = DetModel.evaluate(X_test, Y_test, verbose=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 18        
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________
Train on 18

1800/1800 [==============================] - 0s 75us/sample - loss: 0.0051 - val_loss: 0.0059
Epoch 61/200
1800/1800 [==============================] - 0s 68us/sample - loss: 0.0051 - val_loss: 0.0058
Epoch 62/200
1800/1800 [==============================] - 0s 66us/sample - loss: 0.0051 - val_loss: 0.0057
Epoch 63/200
1800/1800 [==============================] - 0s 66us/sample - loss: 0.0051 - val_loss: 0.0057
Epoch 64/200
1800/1800 [==============================] - 0s 66us/sample - loss: 0.0050 - val_loss: 0.0059
Epoch 65/200
1800/1800 [==============================] - 0s 63us/sample - loss: 0.0050 - val_loss: 0.0061
Epoch 66/200
1800/1800 [==============================] - 0s 70us/sample - loss: 0.0050 - val_loss: 0.0053
Epoch 67/200
1800/1800 [==============================] - 0s 68us/sample - loss: 0.0049 - val_loss: 0.0057
Epoch 68/200
1800/1800 [==============================] - 0s 67us/sample - loss: 0.0049 - val_loss: 0.0051
Epoch 69/200
1800/1800 [==========================

1800/1800 [==============================] - 0s 61us/sample - loss: 0.0017 - val_loss: 0.0017
Epoch 137/200
1800/1800 [==============================] - 0s 64us/sample - loss: 0.0017 - val_loss: 0.0019
Epoch 138/200
1800/1800 [==============================] - 0s 63us/sample - loss: 0.0016 - val_loss: 0.0015
Epoch 139/200
1800/1800 [==============================] - 0s 62us/sample - loss: 0.0016 - val_loss: 0.0013
Epoch 140/200
1800/1800 [==============================] - 0s 62us/sample - loss: 0.0016 - val_loss: 0.0014
Epoch 141/200
1800/1800 [==============================] - 0s 62us/sample - loss: 0.0016 - val_loss: 0.0013
Epoch 142/200
1800/1800 [==============================] - 0s 63us/sample - loss: 0.0015 - val_loss: 0.0011
Epoch 143/200
1800/1800 [==============================] - 0s 63us/sample - loss: 0.0015 - val_loss: 0.0013
Epoch 144/200
1800/1800 [==============================] - 0s 63us/sample - loss: 0.0015 - val_loss: 0.0014
Epoch 145/200
1800/1800 [=================

In [10]:
weights = DropModel.get_weights() # returs a numpy list of weights

In [11]:
Wmat = np.array(weights)

/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [12]:
Wmat

array([array([[ 0.5289524 ,  0.03677871,  0.01636943, -0.49115437,  0.55028903],
       [ 0.06325937,  0.01106238, -0.13638416,  0.04902671,  0.13431966],
       [-0.2916781 , -0.84014183, -0.54846245, -0.27923927, -0.09687316],
       [-0.7185963 ,  0.5405178 , -0.49272394, -0.9201785 ,  0.30906847],
       [-0.53559464,  0.46929413, -0.7488506 , -0.5759799 ,  0.45849922]],
      dtype=float32),
       array([ 0.15500166,  0.307779  ,  0.29716873,  0.09269994, -0.08818484],
      dtype=float32),
       array([[ 0.3995275 , -0.674244  ,  0.5202375 ,  0.42355204, -0.3590286 ],
       [ 0.3636934 , -0.53237474,  0.41512573, -1.5093578 ,  1.439151  ],
       [ 0.5848289 , -0.59625465,  0.73731935, -0.00629348, -0.23045887],
       [ 0.665663  , -0.2830651 , -0.11190548, -0.6265027 , -0.9937679 ],
       [ 0.4187544 ,  0.47960475, -0.45731694, -0.38835138, -0.18239486]],
      dtype=float32),
       array([ 0.19006649,  0.07971375, -0.06191228,  0.16952942, -0.00212534],
      dtype=float3

In [13]:
import pickle as pkl
fileObject = open("weights_initialization.npy", 'wb')
pkl.dump(Wmat, fileObject)
fileObject.close()

#file = open("weights_init.npy", "wb")
#np.save(file, Wmat)
#file.close

In [14]:
def predictive_model_resnet(zk,Wmat): 
    #Assuming you can pass extra arguments to the function for casadi, otherwsise
    #Number of hidden layers and nodes are given once when we get the optimal resnet
    #The Bayesian Optimization Loop will be changing 
    n_hidden_layers = 3
    n_nodes = 5
    model_input = zk.reshape(1,5)
    Wmat_shape = Wmat.shape[0]
    
    #Initialize from input 
    sum0 = np.matmul(zk,Wmat[0]) #Matrix multiplication weights*inputs
    sumbias0 = sum0 + Wmat[1] #Add bias to construct y = sum(wi*xi) + b
    #neuron_out = swish_eval(sum1bias)
    neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation, out = fact(y)
    #They are vector quantitites so you might need to change stuff a bit with symbolic vars
    print(neuron_out.shape)

    #Forward prediction in hidden layers
    for k in range(n_hidden_layers-1):
        sum0 = np.matmul(neuron_out,Wmat[2*k+2])
        sumbias0 = sum0 + Wmat[2*k+3]
        neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation

    #Final output layer with linear activation
    sum0 = np.matmul(neuron_out,Wmat[Wmat_shape-2])
    sumbias0 = sum0 + Wmat[Wmat_shape-1]
    neuron_out = sumbias0  #linear activation

    return neuron_out #+ model_input[0,2:5]

def plantDynamics(matlabEngine, xAll, z):
    # Define the parameters whose scope is limited to this function
    p1 = 82.3
    p2 = 0.5
    p3 = 0.9
    
    # Convert np arrays to list and then to matlab.double
    xMatlab = matlab.double(xAll.tolist())
    zMatlab = [matlab.double([z[0,0]]), matlab.double([z[0,1]])]
    
    # Call the BCR_Discrete function and convert it to np.array since it is matlab type intially
    xNext = np.array(matlabEngine.BCR_Discrete(xMatlab, zMatlab[0], zMatlab[1], p1, p2, p3))
    
    return xNext

In [15]:
zk=Zin[0,:].reshape(1,5)

In [16]:
zkplusone  = predictive_model_resnet(zk,Wmat)

(1, 5)


In [17]:
zk[0,0]

0.20933663257666799

In [18]:
print('Current State:', zk,'Future State', zkplusone)

Current State: [[0.20933663 0.34245236 0.3        0.04       0.02      ]] Future State [[0.30010579 0.04284709 0.01470639]]


In [19]:
initial_cond_plant =scipy.io.loadmat('init_cond.mat')

In [20]:
xo = initial_cond_plant.get("yo")

In [21]:
xo.shape

(1, 87)

In [22]:
x0 = xo[0,-3:]

For BCR_Discrete you must convert the states array to a list like below and then put it into matlab.double; \
Same for matlab.double for the scalars \
The other, fixed parameters are ok like that

In [23]:
zkplusone_plant = eng.BCR_Discrete(matlab.double(xo.tolist()),matlab.double([zk[0,0]]),matlab.double([zk[0,1]]),82.3,1.0,0.9)
zkfuture_plant = np.array(zkplusone_plant)
print(zkfuture_plant[0,-3:]) #Getting the last states

[0.32873736 0.04836853 0.02166765]


In [24]:
zz = plantDynamics(eng, xo, zk)
print(zz[0,-3:])

[0.31459814 0.04428607 0.02081285]


In [25]:
zkfuture_plant  = predictive_model_resnet(np.array([zk[0,0],zk[0,1],x0[0],x0[1],x0[2]]),Wmat)

(5,)


In [26]:
print(zk)
print(zkfuture_plant)

[[0.20933663 0.34245236 0.3        0.04       0.02      ]]
[0.30010579 0.04284709 0.01470639]


One of the issues here with the zkfuture_plant

When you are at the initial condition, or acetate and ethanol are very low, usually you will predict negative concentrations for acetate and ethanol. In other words, you will predict x[1], and x[2] <0 which is unphysical. In this case you can make the prediction equal to zero (because otherwise you might not get feasible solutions)

#Objective Function
Area = 0.002436;   % reactor cross-sectional area, m^2 \
L = 1.06;   % reactor length, m  \
Qmedia[j] = Area \times L \times Dk[j]  (Dk is u[0], i.e., dilution rate) \
Try to maximize sum (Qmedia[j]*x1[j] ) (m3/h * mmol/m^3)  \
Constraints:
Selectivity is defined as Ce/Ca or x2/x1 and I want it to be x2/x1 < 0.45 \
u[0] in (0.01, 0.25) \
u[1] in (0.0, 0.9) \ 
x0,x1,x2 > 0 
